# Job descriptions

Given a particular company's career page:
- Download+cache all open job descriptions
- Feed all job descriptions through an LLM extractor
- Standardize the job description information

Experiment ideas:
- Try using a Langsmith dataset and experiment
- Try building out tools for the popular ATS systems to best handle their data formats
- Extend the scraping code to pick up a career page dynamically and shift to ATS-specific parsers


In [1]:
from core import init

init()

In [1]:
# try testing it out on Gable
from data_sources.job_listings import Workable, SmartRecruiters, Ashby

gable_listings = await Workable.crawl_jobs("https://apply.workable.com/gable/")

gable_listings

In [2]:
gable_results = [Workable.parse_job_response(job) for job in gable_listings]
gable_results

[{'url': 'https://apply.workable.com/gable/j/99A4F53602/',
  'job_description_text': "Share this job\xa0SVGs not supported by this browser.DescriptionAbout Us: Gable.ai is a Seattle-based startup revolutionizing the data industry. Through our data communication, change management, and collaboration platform, we empower developers to build and manage data assets, bridging the gap between data producers and consumers to upscale data quality. Fresh out of stealth mode and backed by prominent venture partners, our mission is to reshape data management by fostering collaboration and innovation. Join us in transforming the landscape of the data industry!Responsibilities:As a Senior Software Engineer, your primary responsibilities will be to own design and delivery of full stack product features. A key component of your work will be to scope out and build the best-possible data solutions to green-field problems at scale. This includes working directly with customers to define the problem, sho

In [ ]:
abridge_listings = await Ashby.crawl_jobs("https://jobs.ashbyhq.com/Abridge")
abridge_listings

In [ ]:
logic2020_listings = await SmartRecruiters.crawl_jobs("https://jobs.smartrecruiters.com/Logic2020")

In [ ]:

# import pandas as pd

# job_descriptions = [
#     parse_smartrecruiters_response(response) for response in sr_jd_responses
# ] + [
#     parse_workable_response(response) for response in jd_responses if not isinstance(response, ScrapflyError)
# ] + [
#     parse_ashby_response(response) for response in ashby_jd_responses if not isinstance(response, ScrapflyError)
# ]

# data = pd.DataFrame(job_descriptions)
# data

,url,job_description
0,https://jobs.smartrecruiters.com/Logic2020Inc/...,Senior Consultant - Energy & UtilitiesFull-tim...
1,https://jobs.smartrecruiters.com/Logic2020Inc/...,Consultant - Strategy & OperationsFull-timePra...
2,https://jobs.smartrecruiters.com/Logic2020Inc/...,Consulting Manager - Energy & UtilitiesFull-ti...
3,https://jobs.smartrecruiters.com/Logic2020Inc/...,Consulting Manager - Energy & UtilitiesFull-ti...
4,https://jobs.smartrecruiters.com/Logic2020Inc/...,Manager - SAP S/4HANA Functional AnalystFull-t...
5,https://jobs.smartrecruiters.com/Logic2020Inc/...,Sr. Business Development ExecutiveFull-timePra...
6,https://jobs.smartrecruiters.com/Logic2020Inc/...,Sr. Business Development ExecutiveFull-timePra...
7,https://jobs.smartrecruiters.com/Logic2020Inc/...,Sr. Business Development ExecutiveFull-timePra...
8,https://jobs.smartrecruiters.com/Logic2020Inc/...,Senior Business Development ExecutiveFull-time...
9,https://apply.workable.com/gable/j/1A48FFBF3B/,Share this job SVGs not supported by this brow...


In [ ]:
# data.to_csv("scraped_job_descriptions.csv", index=False)

In [ ]:
# from markdownify import markdownify

# example_html = BeautifulSoup(sr_jd_responses[0].content).find("main").prettify()

# print(markdownify(example_html, heading_style="atx"))


# Senior Consultant - Energy & Utilities


* Full-time




* Practice Area: Grid Operations
* Level: IV
* Years of Experience: 2-4

## Company Description


We’re a eight-time “Best Company to Work For,” where intelligent, talented people come together to do outstanding work—and have a lot of fun while they’re at it. We offer a solution-focused environment full of collaboration and dedication, to our goals and to each other. You’ll have the opportunity to drive your own success in a supportive, globally connected environment. From advanced tools and technology to an immersive company culture, working at Logic20/20 means working on the leading edge, with a community of the right people around you.



## Job Description


* **Process Engineering:**
  + Conduct in-depth process analysis and assessments to identify inefficiencies, bottlenecks, and opportunities for improvement.
  + Develop and document detailed process maps, workflows, and procedures.
  + Design and implement process opti